In [22]:
%reset -f
import os
import shutil
import sys

import numpy as np
from scipy import sparse

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sn
sn.set()

import pandas as pd

import tensorflow as tf
from tensorflow.contrib.layers import apply_regularization, l2_regularizer

import bottleneck as bn

In [23]:
### change `DATA_DIR` to the location where movielens-20m dataset sits
DATA_DIR = '/Users/benjamin/Downloads/ml-20m'

raw_data = pd.read_csv(os.path.join(DATA_DIR, 'ratings.csv'), header=0)


In [24]:
# load CIL data which contains a list of Ids and Predictions
#  - Id consists of rXX_cYY where XX is the row number (= user id)
#    and YY is the col number (= movie id)
#
# We need to format this to 3 cols: userID (=XX), movieID (=YY) and
# the rating (= Prediction). Should be sorted by userID and movieID.

CIL_DATA_DIR = "/Users/benjamin/FS18/cil/CILRecommender2018/data/"
#training_data = 'trainingIndices.csv'
training_data = 'data_train.csv'
raw_data_2 = pd.read_csv(os.path.join(CIL_DATA_DIR, training_data), header=0)

In [61]:
raw_data_small = raw_data_2
pdata = raw_data_small['Id'].apply(lambda x: pd.Series(x.split('_')))
pdata['rating'] = raw_data_small['Prediction']
pdata.rename(columns={0:'userId',1:'movieId'},inplace=True)
pdata['userId'] = pdata['userId'].str.replace('[^0-9]', '')
pdata['movieId'] = pdata['movieId'].str.replace('[^0-9]', '')
#pdata['userId'].astype('int32', copy=False)
pdata['userId'] = pdata['userId'].astype('int32', copy=True)
pdata['movieId'] = pdata['movieId'].astype('int32', copy=True)
pdata = pdata.sort_values(by=['userId', 'movieId'])
pdata = pdata.reset_index(drop=True)
pdata.to_csv(os.path.join(CIL_DATA_DIR, 'data_train_VAE.csv'))

In [62]:
# create file containing indices of the user/item pairs we want to predict


,userId,movieId,rating
1176947,10000,986,1
1176948,10000,991,2
1176949,10000,996,3
1176950,10000,997,3
1176951,10000,1000,3


In [14]:
# 

,userId,movieId,rating,timestamp
20000258,138493,68954,4.5,1258126920
20000259,138493,69526,4.5,1259865108
20000260,138493,69644,3.0,1260209457
20000261,138493,70286,5.0,1258126944
20000262,138493,71619,2.5,1255811136
